# Experimenting with RQ

Experiments using a Redis server to create job queues. Redis must be running locally on the same machine that's hosting the Jupyter server. Moreover, the `redis` and `rq` Python libraries must be installed (`pip` does the job).

See the readme for further info.

In [ ]:
from redis import Redis
from rq import Queue, Worker
import tasks
from time import sleep

Instantiate a `Redis()` object to connect to the Redis database.

In [ ]:
redis = Redis()

In [ ]:
redis.info()

Instantiate a Queue object connected to the Redis database running locally.

In [ ]:
queue = Queue(connection=redis)

Send a job to the queue ("enqueue"). `queue.enqueue()` returns a `Job` object for which an ID is generated: this will be the key to retrieve all the information about the job from the Redis database.

To check the status of the job, call its `status` attribute. The initial status is `'started'`, whcih changes to `'finished'` if the worker exited succesfully after executing the fucntion or to `'failed'` if the worker exited with an error.

The result of a job can be obtained calling its `result` attribute, which will be None in case the job is still running or has failed.

In [ ]:
job = queue.enqueue(tasks.slow_multiply, 2, 3)

In [ ]:
while job.status=='started':
    print("Computing...")
    sleep(1)
    
print("New status:", job.status)
print("Result:", job.result)

List all the jobs in the queue: these are the jobs that are waiting to be executed by a worker: as soon as a worker starts one of the job in the queue, it disappears from the queue itself.

In [ ]:
queue.get_jobs()

Get all the jobs keys from the Redis database.

In [ ]:
redis.keys()

Get the workers. If no worker is currently running (to instantiate one, run `rq worker` in the terminal), this will just return an empty list.

Notice that the worker must be run in the directory that contains the Python module in which the functions being called is defined.

In [ ]:
workers = Worker.all(connection=Redis())
workers